In [14]:
import graph_tool.all as gt
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
def run_loop(g):
    while True:
        print('\n========= OPTIONS =========')
        print('[0]: Print graph')
        print('[1]: Print blockmodel graph')
        print('[q]: Quit')
        
        choice = input('\nEnter the number of what you\'d like to do: ')

        match choice:
            case '0':
                gt.graph_draw(g)
            case '1':
                state = gt.minimizworm_blockmodel_dl(g)
                state.draw()
            case 'q':
                print('\nQuitting...')
                break
            case _:
                print('\nInvalid entry, try again.')

def average_shortest_path_length(g):
    """
    Calculate the average shortest path length using graph-tool.

    Parameters:
        g (Graph): A graph-tool Graph object.

    Returns:
        float: Average shortest path length.
    """
    # Compute all-pairs shortest distances
    total_length = 0
    total_pairs = 0

    distances = gt.shortest_distance(g, weights=g.ep.value).a

    print(distances)

    for v in g.vertices():
        distances = gt.shortest_distance(g, source=v, weights=g.ep.value).a  # Get as numpy array
        finitworm_distances = distances[distances < float("inf")]
        total_length += finitworm_distances.sum()
        total_pairs += len(finitworm_distances) - 1  # Exclude self-distances

    if total_pairs > 0:
        return total_length / total_pairs
    else:
        return float("inf")

def print_summmary_statistics(g):
    dist_unweighted, ends_unweighted = gt.pseudo_diameter(g)
    dist_weighted, ends_weighted =  gt.pseudo_diameter(g, weights=g.ep.value)
    c, num_triangles, num_triples = gt.global_clustering(g, weight=g.ep.value, ret_counts=True)
    
    print('Pseudo-diameter: (weighted):', dist_weighted)
    print('Average shortest path length:', average_shortest_path_length(g))
    print('Clustering coefficient:', c[0], 'with standard deviation', c[1])
    print('Number of triangles:', num_triangles)
    print('Number of triples:', num_triples)

def print_motif_statistics(g):
    num_v_in_motif = 3
    motifs, num_motifs = gt.motifs(g, num_v_in_motif)
    print('Number of motifs:', num_motifs)

def is_graph_connected(edges, num_nodes):
    """ c
    Check if a graph is connected.

    Parameters:
        edges (list of tuples): List of (source, target, weight) edges.
        num_nodes (int): Total number of nodes in the graph.

    Returns:
        bool: True if the graph is connected, False otherwise.
    """
    # Build adjacency list
    adjacency_list = defaultdict(list)
    for src, tgt, weight in edges:
        adjacency_list[src].append(tgt)
        adjacency_list[tgt].append(src)

    visited = set()

    def dfs(node):
        """Depth-First Search to visit nodes."""
        visited.add(node)
        for neighbor in adjacency_list[node]:
            if neighbor not in visited:
                dfs(neighbor)

    # Start DFS from the first node
    dfs(0)

    # Check if all nodes are visited
    return len(visited) == num_nodes

def print_vertex_statistics(v):
    k_in = v.in_degree()
    k_out = v.out_degree()
    k_tot = k_in + k_out
    print('=== Vertex Statistics ===')
    print(f'Total Degree:', k_tot)
    print(f'In Degree:', k_in)
    print(f'Out Degree:', k_out)

def plot_degree_distribution(g, degree_type="in", weighted=False, flag=0):
    """
    Plot the degree distribution of a graph.

    Parameters:
    g: graph_tool.Graph
        The graph object.
    degree_type: str
        "in" for in-degrees, "out" for out-degrees.
    weighted: bool
        True for weighted degree distribution, False for unweighted.
    flag: bool
        Print if 1
    """
    # Determine degree calculation based on parameters
    if degree_type == "in":
        if weighted:
            degrees = g.get_in_degrees(list(g.vertices()), g.ep.value)
            title = "In-degree distribution (weighted)"
        else:
            degrees = g.get_in_degrees(list(g.vertices()))
            title = "In-degree distribution (unweighted)"
    elif degree_type == "out":
        if weighted:
            degrees = g.get_out_degrees(list(g.vertices()), g.ep.value)
            title = "Out-degree distribution (weighted)"
        else:
            degrees = g.get_out_degrees(list(g.vertices()))
            title = "Out-degree distribution (unweighted)"
    else:
        raise ValueError("degree_type must be either 'in' or 'out'.")

    # Create histogram
    if flag == 1:
        counts, bins = np.histogram(degrees)
        plt.figure(figsize=(10, 5))
        plt.stairs(counts, bins)
        plt.xlabel('Degree (k)')
        plt.ylabel('Frequency')
        plt.title(title)
        plt.show()

    return degrees

def print_basic_statistics(g, flag):
    n = g.num_vertices()
    m = g.num_edges()

    k_ins_unweighted = plot_degree_distribution(g, degree_type='in', weighted=False) # add 'flag=1' as a parameter to plot a histogram
    k_ins_weighted = plot_degree_distribution(g, degree_type='in', weighted=True) # add 'flag=1' as a parameter to plot a histogram
    k_outs_unweighted = plot_degree_distribution(g, degree_type='out', weighted=False) # add 'flag=1' as a parameter to plot a histogram
    k_outs_weighted = plot_degree_distribution(g, degree_type='out', weighted=True) # add 'flag=1' as a parameter to plot a histogram

    print('Number of nodes:', n, '\n',
          'Number of edges:', m, '\n',
          'Mean degree (unweighted):', np.mean(k_ins_unweighted), '\n',
          'Mean degree (weighted)', np.mean(k_ins_weighted), '\n')

    print('=== IN DEGREE ===', '\n',
          '=== UNWEIGHTED ===', '\n',
          'Maximum:', max(k_ins_unweighted), '\n',
          'Minimum:', min(k_ins_unweighted), '\n',
          '=== WEIGHTED ===', '\n',
          'Maximum:', max(k_ins_weighted), '\n',
          'Minimum:', min(k_ins_weighted), '\n')
    
    print('=== OUT DEGREE ===', '\n',
          '=== UNWEIGHTED ===', '\n',
          'Maximum:', max(k_outs_unweighted), '\n',
          'Minimum:', min(k_outs_unweighted), '\n',
          '=== WEIGHTED ===', '\n',
          'Maximum:', max(k_outs_weighted), '\n',
          'Minimum:', min(k_outs_weighted), '\n')
    
    
    # print('=== TOTAL DEGREE ===', '\n',
    #       'UNWEIGHTED', '\n',
    #       'Mean:', np.mean(k_ins_unweighted), '\n',
    #       'Maximum:', max(k_ins_unweighted), '\n',
    #       'Minimum:', min(k_ins_unweighted), '\n',
    #       '\n',
    #       'WEIGHTED', '\n',
    #       'Mean:', np.mean(k_ins_weighted), '\n',
    #       'Maximum:', max(k_ins_weighted), '\n',
    #       'Minimum:', min(k_ins_weighted), '\n'))

In [17]:
g = gt.collection.data["celegansneural"]

In [18]:
distances = gt.shortest_distance(g)

In [20]:
print(distances)
print(list(distances))

<VertexPropertyMap object with value type 'vector<int32_t>', for Graph 0x14a512ae0, at 0x149a4cd40>
[array([         0,          1,          1,          1,          1,
                1,          1,          1,          1,          1,
                2,          2,          2,          2,          2,
                3,          3,          4,          3,          3,
                3,          3,          3,          2,          2,
                2,          4,          2,          3,          3,
                3,          3,          3,          3,          2,
                2,          2,          3,          3,          3,
                4,          4,          3,          2,          3,
                3,          2,          2,          2,          3,
                2,          3,          3,          5, 2147483647,
                3,          4, 2147483647,          3,          2,
                4,          4,          4,          3,          3,
                3,          

In [ ]:

n = g.num_vertices()
m = g.num_edges()
k = m/n
print(n)
print(m)
print(k)


connectivity_probability = m / (n *(n-1))

print(connectivity_probability)

297
2359
7.942760942760943
0.026833651833651835
